In [183]:
#dataset sourced by an individual csgo player from https://www.kaggle.com/thesiff/counterstrike
#simple analysis of win/loss data

import numpy as np
import pandas as pd
df = pd.read_csv('CSGOComplete.csv')
df

,Map,Day,Month,Year,Date,Wait Time(s),Match Time sec,Team A Rounds,Team B Rounds,Ping,Kills,Assists,Deaths,Mvp's,HS%,Points,Result
0,Mirage,3,8,2018,03-08-2018,327,2906,16,13,215,17,2,21,2,5,45,Win
1,Mirage,2,8,2018,02-08-2018,336,2592,16,11,199,13,4,24,2,0,40,Lost
2,Mirage,31,7,2018,31-07-2018,414,2731,16,14,85,15,3,18,3,26,37,Win
3,Mirage,31,7,2018,31-07-2018,317,2379,11,16,93,12,2,15,2,16,30,Lost
4,Mirage,30,7,2018,30-07-2018,340,3467,15,15,94,33,5,20,5,30,83,Tie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Dust II,23,7,2015,23-07-2015,2,1573,16,4,46,12,2,20,0,16,26,Lost
1129,Dust II,23,7,2015,23-07-2015,29,2126,16,8,41,19,6,21,2,31,51,Lost
1130,Dust II,23,7,2015,23-07-2015,10,2555,11,16,17,9,3,20,2,55,30,Lost
1131,Dust II,23,7,2015,23-07-2015,9,2293,8,16,20,11,4,20,1,27,31,Lost


In [184]:
newdf = df.drop(columns = ['Map','Day','Month','Year','Date','Wait Time(s)'])
newdf = newdf.drop(columns = ['Team A Rounds','Team B Rounds','Kills','Assists','Deaths','Mvp\'s','HS%','Points'])
newdf = newdf.sort_values(['Ping'])
newdf

,Match Time sec,Ping,Result
671,1899,0,Win
89,2752,0,Tie
746,1529,0,Lost
341,2291,0,Win
1057,2280,0,Lost
...,...,...,...
283,2739,422,Lost
339,2466,449,Lost
29,2900,563,Win
18,1500,632,Win


In [185]:
winner = newdf.loc[(newdf['Ping'] >= 150) & (newdf['Result'] == 'Win')]
loser = newdf.loc[(newdf['Ping'] >= 150) & (newdf['Result'] == 'Lost')]
print('+ Average match time = {} minutes'.format(int(newdf['Match Time sec'].mean()/60)))
a = newdf.loc[(newdf['Match Time sec'] >= 2280) & (newdf['Result'] == 'Win')].shape[0]
b = newdf.loc[(newdf['Match Time sec'] <= 2280) & (newdf['Result'] == 'Win')].shape[0]
print('+ Wins if time greater than 38 minutes = {}'.format(a))
print('+ Wins if time less than 38 minutes = {}'.format(b))
print('+ The ratio {0:.4f} concludes that length of match does not have a big impact on win/loss'.format(a/b))

+ Average match time = 38 minutes
+ Wins if time greater than 38 minutes = 232
+ Wins if time less than 38 minutes = 256
+ The ratio 0.9062 concludes that length of match does not have a big impact on win/loss


In [186]:
total = loser.shape[0] + winner.shape[0]
winratio = winner.shape[0]/total
print('+ Chances of winning with ping greater than 150ms = {0:.4f}%'.format(winratio*100))

+ Chances of winning with ping greater than 150ms = 39.0625%


In [187]:
win150 = newdf.loc[(newdf['Ping'] <= 150) & (newdf['Result'] == 'Win')]
loss150 = newdf.loc[(newdf['Ping'] <= 150) & (newdf['Result'] == 'Lost')]
total150 = loss150.shape[0] + win150.shape[0]
winratio150 = win150.shape[0]/total150
print('+ Chances of winning with ping less than 150ms = {0:.4f}%'.format(winratio150*100))

+ Chances of winning with ping less than 150ms = 47.0051%


In [188]:
mvpg2 = df.loc[(df['Mvp\'s']>2) & (df['Result'] == 'Win')].shape[0]
mvpl2 = df.loc[(df['Mvp\'s']<2) & (df['Result'] == 'Win')].shape[0]
ratio = mvpg2 / mvpl2
print('+ Winning ratio if you mvp\'s were greater than 2 = {0:.2f}'.format(ratio))

+ Winning ratio if you mvp's were greater than 2 = 1.22


In [189]:
print('+ Conclusion:\n** There\'s a good chance that you will lose 5 matches out of 10 regularly if you got a fair competition.\n** Ping does matter but would make you win just 1 more match if you\'ve the upper side with ping and similar competition.')

+ Conclusion:
** There's a good chance that you will lose 5 matches out of 10 regularly if you got a fair competition.
** Ping does matter but would make you win just 1 more match if you've the upper side with ping and similar competition.


In [190]:
ntie = newdf.loc[(newdf['Result'] == 'Tie')].shape[0]
ratiotie = ntie / newdf.shape[0]
print('+ Also tie percentage: {0:.4f}%'.format(ratiotie*100))

+ Also tie percentage: 7.5022%
